In [1]:
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import *
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

In [2]:
#Setting Up GPU Growth. Helps getting rid of out of memory errors
physical_devices=tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs available: ", len(physical_devices))
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

Num GPUs available:  1


In [3]:
embed=load_model('embed.h5')

In [4]:
def define_discriminator(in_shape=(28,28,1),n_classes=5):
    """
    #label input
    in_label=Input(shape=(1,))
    li=Embedding(n_classes,50)(in_label)#Embedding for categorical input
    n_nodes=in_shape[0]*in_shape[1]#scaling up tp img dimension with linear activision
    li=Dense(n_nodes)(li)
    li=Reshape((in_shape[0],in_shape[1],1))(li)#Reshape to additional channel
    """
    
    #label input
    in_label=Input(shape=(2,))
    li=Dense(50)(in_label)
    n_nodes=in_shape[0]*in_shape[1]#scaling up tp img dimension with linear activision
    li=Dense(n_nodes)(li)
    li=Reshape((in_shape[0],in_shape[1],1))(li)#Reshape to additional channel
    
    #image input
    in_image=Input(shape=(in_shape))
    merge=Concatenate()([in_image,li])#concat label as a channel
    #downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
    

In [5]:
def define_generator(latent_dim,n_classes=5):
   
    #label input
    in_label=Input(shape=(2,))
    li=Dense(50)(in_label)
    n_nodes=7*7
    li=Dense(n_nodes)(li)
    li=Reshape((7,7,1))(li)
    
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output
    out_layer = Conv2D(1, (7,7), activation='relu', padding='same')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [7]:
# load fashion mnist images
def load_real_samples():
    # load dataset
    with open('data.npy', 'rb') as f:
        X = np.load(f)
    X=np.reshape(X,(-1,28,28,1))
    y=embed.predict(x=X)
    return[X,y]

In [8]:
# # select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples,dataset):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    images, labels = dataset
    ix = randint(0, images.shape[0], n_samples)
    labels = labels[ix]
    return [z_input, labels]

In [10]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples,dataset):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples,dataset)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

In [11]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    d1=[]
    d2=[]
    g=[]
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            # generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch,dataset)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            # prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch,dataset)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
            d1.append(d_loss1)
            d2.append(d_loss2)
            g.append(g_loss)
    # save the generator model
    g_model.save('CGAN.h5')
    with open('d_loss1.npy','wb') as f:
        np.save(f,np.array(d1))
    with open('d_loss2.npy','wb') as f:
        np.save(f,np.array(d2))
    with open('g_loss.npy', 'wb') as f:
        np.save(f, np.array(g))

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

2/2 [==============================] - 0s 4ms/step
>1, 1/546, d1=0.654, d2=0.716 g=0.684
2/2 [==============================] - 0s 3ms/step
>1, 2/546, d1=0.447, d2=0.927 g=0.555
2/2 [==============================] - 0s 4ms/step
>1, 3/546, d1=0.352, d2=1.608 g=0.426
2/2 [==============================] - 0s 4ms/step
>1, 4/546, d1=0.431, d2=1.837 g=0.712
2/2 [==============================] - 0s 3ms/step
>1, 5/546, d1=0.612, d2=0.474 g=2.372
2/2 [==============================] - 0s 3ms/step
>1, 6/546, d1=0.770, d2=0.323 g=1.661
2/2 [==============================] - 0s 3ms/step
>1, 7/546, d1=0.812, d2=0.576 g=0.850
2/2 [==============================] - 0s 4ms/step
>1, 8/546, d1=0.816, d2=0.769 g=0.656
2/2 [==============================] - 0s 3ms/step
>1, 9/546, d1=0.683, d2=0.781 g=0.646
2/2 [==============================] - 0s 3ms/step
>1, 10/546, d1=0.604, d2=0.741 g=0.670
2/2 [==============================] - 0s 4ms/step
>1, 11/546, d1=0.494, d2=0.713 g=0.690
2/2 [==============

2/2 [==============================] - 0s 3ms/step
>1, 91/546, d1=0.654, d2=0.711 g=0.770
2/2 [==============================] - 0s 3ms/step
>1, 92/546, d1=0.594, d2=0.739 g=0.754
2/2 [==============================] - 0s 3ms/step
>1, 93/546, d1=0.613, d2=0.721 g=0.744
2/2 [==============================] - 0s 4ms/step
>1, 94/546, d1=0.571, d2=0.738 g=0.787
2/2 [==============================] - 0s 3ms/step
>1, 95/546, d1=0.573, d2=0.673 g=0.815
2/2 [==============================] - 0s 3ms/step
>1, 96/546, d1=0.575, d2=0.668 g=0.841
2/2 [==============================] - 0s 3ms/step
>1, 97/546, d1=0.594, d2=0.676 g=0.839
2/2 [==============================] - 0s 3ms/step
>1, 98/546, d1=0.577, d2=0.674 g=0.839
2/2 [==============================] - 0s 3ms/step
>1, 99/546, d1=0.621, d2=0.651 g=0.804
2/2 [==============================] - 0s 4ms/step
>1, 100/546, d1=0.578, d2=0.690 g=0.822
2/2 [==============================] - 0s 4ms/step
>1, 101/546, d1=0.546, d2=0.685 g=0.801
2/2 [===

2/2 [==============================] - 0s 3ms/step
>1, 181/546, d1=0.632, d2=0.679 g=0.891
2/2 [==============================] - 0s 3ms/step
>1, 182/546, d1=0.708, d2=0.674 g=0.921
2/2 [==============================] - 0s 3ms/step
>1, 183/546, d1=0.705, d2=0.699 g=0.880
2/2 [==============================] - 0s 3ms/step
>1, 184/546, d1=0.728, d2=0.728 g=0.877
2/2 [==============================] - 0s 3ms/step
>1, 185/546, d1=0.719, d2=0.699 g=0.965
2/2 [==============================] - 0s 4ms/step
>1, 186/546, d1=0.758, d2=0.635 g=0.993
2/2 [==============================] - 0s 4ms/step
>1, 187/546, d1=0.792, d2=0.602 g=0.942
2/2 [==============================] - 0s 3ms/step
>1, 188/546, d1=0.788, d2=0.638 g=1.001
2/2 [==============================] - 0s 3ms/step
>1, 189/546, d1=0.764, d2=0.638 g=0.945
2/2 [==============================] - 0s 4ms/step
>1, 190/546, d1=0.748, d2=0.652 g=0.921
2/2 [==============================] - 0s 3ms/step
>1, 191/546, d1=0.731, d2=0.604 g=0.906

2/2 [==============================] - 0s 3ms/step
>1, 271/546, d1=0.677, d2=0.700 g=0.836
2/2 [==============================] - 0s 3ms/step
>1, 272/546, d1=0.640, d2=0.666 g=0.820
2/2 [==============================] - 0s 4ms/step
>1, 273/546, d1=0.661, d2=0.710 g=0.783
2/2 [==============================] - 0s 3ms/step
>1, 274/546, d1=0.655, d2=0.760 g=0.795
2/2 [==============================] - 0s 4ms/step
>1, 275/546, d1=0.666, d2=0.718 g=0.920
2/2 [==============================] - 0s 4ms/step
>1, 276/546, d1=0.753, d2=0.635 g=0.986
2/2 [==============================] - 0s 3ms/step
>1, 277/546, d1=0.732, d2=0.615 g=0.934
2/2 [==============================] - 0s 3ms/step
>1, 278/546, d1=0.721, d2=0.677 g=0.855
2/2 [==============================] - 0s 4ms/step
>1, 279/546, d1=0.670, d2=0.639 g=0.858
2/2 [==============================] - 0s 3ms/step
>1, 280/546, d1=0.701, d2=0.658 g=0.877
2/2 [==============================] - 0s 3ms/step
>1, 281/546, d1=0.677, d2=0.632 g=0.869

2/2 [==============================] - 0s 3ms/step
>1, 361/546, d1=0.636, d2=0.631 g=0.936
2/2 [==============================] - 0s 3ms/step
>1, 362/546, d1=0.557, d2=0.563 g=0.952
2/2 [==============================] - 0s 3ms/step
>1, 363/546, d1=0.642, d2=0.599 g=0.966
2/2 [==============================] - 0s 3ms/step
>1, 364/546, d1=0.540, d2=0.620 g=0.865
2/2 [==============================] - 0s 3ms/step
>1, 365/546, d1=0.581, d2=0.779 g=0.811
2/2 [==============================] - 0s 4ms/step
>1, 366/546, d1=0.618, d2=0.784 g=0.815
2/2 [==============================] - 0s 3ms/step
>1, 367/546, d1=0.608, d2=0.682 g=1.152
2/2 [==============================] - 0s 3ms/step
>1, 368/546, d1=0.710, d2=0.515 g=1.215
2/2 [==============================] - 0s 3ms/step
>1, 369/546, d1=0.803, d2=0.653 g=0.815
2/2 [==============================] - 0s 3ms/step
>1, 370/546, d1=0.667, d2=0.715 g=0.881
2/2 [==============================] - 0s 3ms/step
>1, 371/546, d1=0.663, d2=0.635 g=1.032

...

2/2 [==============================] - 0s 3ms/step
>37, 186/546, d1=0.000, d2=0.000 g=17.471
2/2 [==============================] - 0s 3ms/step
>37, 187/546, d1=0.000, d2=0.000 g=17.785
2/2 [==============================] - 0s 6ms/step
>37, 188/546, d1=0.000, d2=0.000 g=17.667
2/2 [==============================] - 0s 4ms/step
>37, 189/546, d1=0.000, d2=0.000 g=17.971
2/2 [==============================] - 0s 3ms/step
>37, 190/546, d1=0.000, d2=0.000 g=17.570
2/2 [==============================] - 0s 3ms/step
>37, 191/546, d1=0.000, d2=0.000 g=17.531
2/2 [==============================] - 0s 4ms/step
>37, 192/546, d1=0.000, d2=0.000 g=17.557
2/2 [==============================] - 0s 4ms/step
>37, 193/546, d1=0.000, d2=0.000 g=17.882
2/2 [==============================] - 0s 3ms/step
>37, 194/546, d1=0.000, d2=0.000 g=17.615
2/2 [==============================] - 0s 3ms/step
>37, 195/546, d1=0.000, d2=0.000 g=17.745
2/2 [==============================] - 0s 3ms/step
>37, 196/546, d1=0.

2/2 [==============================] - 0s 4ms/step
>37, 274/546, d1=0.000, d2=0.000 g=18.331
2/2 [==============================] - 0s 4ms/step
>37, 275/546, d1=0.000, d2=0.000 g=17.582
2/2 [==============================] - 0s 3ms/step
>37, 276/546, d1=0.000, d2=0.000 g=17.823
2/2 [==============================] - 0s 3ms/step
>37, 277/546, d1=0.000, d2=0.000 g=17.740
2/2 [==============================] - 0s 3ms/step
>37, 278/546, d1=0.000, d2=0.000 g=17.599
2/2 [==============================] - 0s 3ms/step
>37, 279/546, d1=0.000, d2=0.000 g=17.780
2/2 [==============================] - 0s 3ms/step
>37, 280/546, d1=0.000, d2=0.000 g=17.990
2/2 [==============================] - 0s 3ms/step
>37, 281/546, d1=0.000, d2=0.000 g=18.147
2/2 [==============================] - 0s 7ms/step
>37, 282/546, d1=0.000, d2=0.000 g=17.758
2/2 [==============================] - 0s 3ms/step
>37, 283/546, d1=0.000, d2=0.000 g=17.704
2/2 [==============================] - 0s 3ms/step
>37, 284/546, d1=0.

2/2 [==============================] - 1s 126ms/step
>37, 362/546, d1=0.000, d2=0.000 g=17.766
2/2 [==============================] - 1s 270ms/step
>37, 363/546, d1=0.000, d2=0.000 g=17.631
2/2 [==============================] - 1s 249ms/step
>37, 364/546, d1=0.000, d2=0.000 g=17.993
2/2 [==============================] - 0s 114ms/step
>37, 365/546, d1=0.000, d2=0.000 g=17.894
2/2 [==============================] - 0s 39ms/step
>37, 366/546, d1=0.000, d2=0.000 g=18.251
2/2 [==============================] - 0s 38ms/step
>37, 367/546, d1=0.000, d2=0.000 g=17.927
2/2 [==============================] - 0s 33ms/step
>37, 368/546, d1=0.000, d2=0.000 g=17.815
2/2 [==============================] - 0s 36ms/step
>37, 369/546, d1=0.000, d2=0.000 g=17.993
2/2 [==============================] - 0s 41ms/step
>37, 370/546, d1=0.000, d2=0.000 g=17.862
2/2 [==============================] - 1s 49ms/step
>37, 371/546, d1=0.000, d2=0.000 g=17.729
2/2 [==============================] - 0s 35ms/step
>37,

2/2 [==============================] - 1s 307ms/step
>37, 448/546, d1=0.000, d2=0.000 g=17.697
2/2 [==============================] - 1s 307ms/step
>37, 449/546, d1=0.000, d2=0.000 g=18.380
2/2 [==============================] - 1s 385ms/step
>37, 450/546, d1=0.000, d2=0.000 g=18.210
2/2 [==============================] - 1s 311ms/step
>37, 451/546, d1=0.000, d2=0.000 g=18.043
2/2 [==============================] - 1s 326ms/step
>37, 452/546, d1=0.000, d2=0.000 g=17.848
2/2 [==============================] - 1s 368ms/step
>37, 453/546, d1=0.000, d2=0.000 g=17.752
2/2 [==============================] - 1s 298ms/step
>37, 454/546, d1=0.000, d2=0.000 g=17.869
2/2 [==============================] - 1s 300ms/step
>37, 455/546, d1=0.000, d2=0.000 g=18.107
2/2 [==============================] - 1s 440ms/step
>37, 456/546, d1=0.000, d2=0.000 g=17.956
2/2 [==============================] - 1s 294ms/step
>37, 457/546, d1=0.000, d2=0.000 g=18.045
2/2 [==============================] - 1s 346ms/st

2/2 [==============================] - 2s 418ms/step
>37, 534/546, d1=0.000, d2=0.000 g=18.414
2/2 [==============================] - 3s 559ms/step
>37, 535/546, d1=0.000, d2=0.000 g=18.124
2/2 [==============================] - 5s 929ms/step
>37, 536/546, d1=0.000, d2=0.000 g=18.445
2/2 [==============================] - 2s 528ms/step
>37, 537/546, d1=0.000, d2=0.000 g=18.647
2/2 [==============================] - 2s 462ms/step
>37, 538/546, d1=0.000, d2=0.000 g=18.103
2/2 [==============================] - 3s 643ms/step
>37, 539/546, d1=0.000, d2=0.000 g=17.996
2/2 [==============================] - 2s 483ms/step
>37, 540/546, d1=0.000, d2=0.000 g=18.563
2/2 [==============================] - 2s 520ms/step
>37, 541/546, d1=0.000, d2=0.000 g=18.269
2/2 [==============================] - 3s 717ms/step
>37, 542/546, d1=0.000, d2=0.000 g=18.050
2/2 [==============================] - 3s 649ms/step
>37, 543/546, d1=0.000, d2=0.000 g=18.122
2/2 [==============================] - 4s 572ms/st